# Veggie Munger

A quick and dirty notebook for munging the veggie purchase data for TRB Dec 2024 - Nov 2025

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data_dir = Path('data')
veggie_file = data_dir / 'Ayush Vegetable April.xlsx'

In [7]:
veggie_file.stem.split()[-1]

'April'

In [8]:
purchase_data_dict = {}

In [17]:
# for each file in data_dir load the data into a dataframe, and add it to purchase_data_dict with the month as the key
for file in data_dir.glob('*.xlsx'):
    month = file.stem.split()[-1]
    df = pd.read_excel(file)
    purchase_data_dict[month] = df

In [18]:
# for each item in purchase_data_dict, rename the 'Kg' column to the key (month), and drop column 'Total amount'
for month, df in purchase_data_dict.items():
    df.rename(columns={'Kg': month}, inplace=True)
    df.drop(columns=['Total amount'], inplace=True)

In [20]:
purchase_data_dict['June'].head()

,Item,June
0,Carrot,1
1,Cauliflower,0
2,Ginger,1.5
3,Onion,25.5
4,Parvel,0


In [21]:
# merge all dataframes into a single dataframe on 'Item' column, using outer join
merged_df = pd.DataFrame()
for month, df in purchase_data_dict.items():
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='Item', how='outer')

In [32]:
# drop duplicated rows if any
merged_df.drop_duplicates(inplace=True)

In [34]:
# drop rows where 'Item' is NaN
merged_df.dropna(subset=['Item'], inplace=True)

In [36]:
# order the columns of merged_df by month as they appear in a year
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
ordered_columns = ['Item'] + [month for month in month_order if month in merged_df.columns]
merged_df = merged_df[ordered_columns]

In [38]:
# keep only Item and months June to November columns
final_columns = ['Item', 'June', 'July', 'August', 'September', 'October', 'November']
merged_df = merged_df[final_columns]

In [39]:
merged_df

,Item,June,July,August,September,October,November
0,Babycorn,0.5,0.5,0.8,1.1,2,1.25
1,Banana Leaf,50pcs,25pcs,10,15pcs,105,50pcs
2,Basil,4.5,4.5,5.3,4.9,6.35,6
3,Brinjal,0,NaN,0,0.5,2,1.25
4,Capsicum,NaN,NaN,NaN,NaN,NaN,NaN
5,Carrot,1,2.75,2,6,6.5,3.25
6,Cauliflower,0,0,0,0,0,6
7,Cherry Tomato,1.5,3.05,2,2.5,3.5,2.75
8,Coriender Leaf,1.35,2.25,1.1,1.3,2.5,1.5
9,Egg,270pcs,600pcs,600pcs,630pcs,660 pcs,702pcs


In [40]:
merged_df.to_excel('merged_veggie_purchases.xlsx', index=False)